# Day 4: ScoreImportance Compilation with GEPA

**Goal**: Compile ScoreImportance module to improve from 77.5% ±2 accuracy to 85%+  
**Optimizer**: GEPA (primary choice) with budget=40 rollouts  
**Expected Runtime**: 4-6 hours  

## Baseline Performance
- ±2 Accuracy: 77.5% (31/40 seeds)
- MAE: 1.45
- Weakest category: Mundane (17%)
- Target: ±2 Accuracy > 85%

## Setup: Install Dependencies

In [ ]:
# Install required packages
!pip install -q dspy-ai sentence-transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 6.6 MB/s eta 0:00:00


## Mount Google Drive (for file persistence)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory structure if needed
!mkdir -p /content/drive/MyDrive/mini-town/compiled
!mkdir -p /content/drive/MyDrive/mini-town/checkpoints

Mounted at /content/drive


## Upload Files

**Required files**:
1. `scorer_v1.json` - 40 training seeds from Day 3
2. `.env` file with GROQ_API_KEY

Upload these to `/content/` or copy from Google Drive

In [ ]:
# Copy from Google Drive (uncomment if files are in Drive)
!cp /content/drive/MyDrive/mini-town/seeds/scorer_v1.json /content/

# Set API key
import os
from getpass import getpass
os.environ['GROQ_API_KEY'] = getpass('Enter your GROQ_API_KEY: ')

Enter your GROQ_API_KEY: ··········


In [ ]:
# Configure Together.ai
import os
from getpass import getpass

together_key = getpass("Enter your Together.ai API key: ")
os.environ["TOGETHER_API_KEY"] = together_key
print("✅ Together.ai API key set in environment")


Enter your Together.ai API key: ··········
✅ Together.ai API key set in environment


## Configure DSPy with Groq LLM

In [ ]:
# Configure Together.ai LM
import os
import dspy

lm = dspy.LM(
    model="together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    api_key=os.getenv("TOGETHER_API_KEY"),
    temperature=0.3,
    max_tokens=512
)

reflection_lm = dspy.LM(
    model="together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    api_key=os.getenv("TOGETHER_API_KEY"),
    temperature=0.5,
    max_tokens=512
)

# Re-configure DSPy with Together.ai
dspy.settings.configure(lm=lm)

print("✅ DSPy configured with Together.ai")
print("   Model: Meta-Llama-3.1-8B-Instruct-Turbo")


✅ DSPy configured with Together.ai
   Model: Meta-Llama-3.1-8B-Instruct-Turbo


## Define ScoreImportance Signature

In [ ]:
class ScoreImportance(dspy.Signature):
    """Rate how important this observation is for the agent's goals.

    Score 1-10 where:
    - 1-2: Trivial, background noise (e.g., "grass is green")
    - 3-4: Mildly interesting but not actionable
    - 5-6: Relevant to goals, worth remembering
    - 7-8: Directly impacts current plans or goals
    - 9-10: Life-changing, urgent, critical to goals
    """

    observation: str = dspy.InputField(desc="What the agent observed")
    agent_goal: str = dspy.InputField(desc="Agent's current high-level goal")
    agent_personality: str = dspy.InputField(desc="Agent's personality traits")

    reasoning: str = dspy.OutputField(desc="Brief explanation of score")
    score: int = dspy.OutputField(desc="Importance score (1-10)")

print("✅ ScoreImportance signature defined")

✅ ScoreImportance signature defined


## Load and Prepare Seeds

In [ ]:
import json

# Load seeds
with open('scorer_v1.json', 'r') as f:
    seeds_data = json.load(f)

print(f"Loaded {len(seeds_data['seeds'])} seeds")
print(f"Categories: {seeds_data['categories']}")

# Convert to DSPy examples
trainset = []
for seed in seeds_data['seeds']:
    example = dspy.Example(
        observation=seed['observation'],
        agent_goal=seed['agent_goal'],
        agent_personality=seed['agent_personality'],
        score=seed['gold_score'],
        category=seed['category'],  # For analysis
        seed_id=seed['id']  # For tracking
    ).with_inputs("observation", "agent_goal", "agent_personality")
    trainset.append(example)

print(f"✅ Created {len(trainset)} training examples")

# Show sample
print("\nSample example:")
print(f"Observation: {trainset[0].observation}")
print(f"Goal: {trainset[0].agent_goal}")
print(f"Personality: {trainset[0].agent_personality}")
print(f"Gold score: {trainset[0].score}")

Loaded 40 seeds
Categories: {'social': 8, 'environmental': 6, 'goal_relevant': 8, 'emotional': 6, 'mundane': 6, 'edge_cases': 6}
✅ Created 40 training examples

Sample example:
Observation: Alice invited me to a party at 7pm tonight
Goal: Build relationships in the neighborhood
Personality: social, optimistic
Gold score: 8


## Define Importance Metric

In [ ]:
def importance_metric(gold, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Metric for ScoreImportance compilation (GEPA-compatible).

    GEPA requires 5 arguments:
    - gold: The gold/example object with .score attribute
    - pred: The prediction object with .score attribute
    - trace: Execution trace (optional, for debugging)
    - pred_name: Name of the predictor (optional)
    - pred_trace: Prediction trace (optional)

    Returns:
    - float: Score between 0.0 and 1.0
    """
    try:
        # Extract scores from objects
        if hasattr(gold, 'score'):
            gold_score = int(gold.score)
        else:
            gold_score = int(gold)

        if hasattr(pred, 'score'):
            pred_score = int(pred.score)
        else:
            pred_score = int(pred)
    except (ValueError, AttributeError, TypeError):
        return 0.0

    # Clamp predicted score to 1–10
    pred_score = max(1, min(10, pred_score))

    # Calculate error
    error = abs(pred_score - gold_score)

    # Return score based on error (higher is better)
    if error == 0:
        return 1.0      # Exact match
    elif error <= 1:
        return 0.8      # Within ±1
    elif error <= 2:
        return 0.5      # Within ±2
    elif error <= 3:
        return 0.2      # Within ±3
    else:
        return 0.0      # Poor prediction


print("✅ GEPA-compatible importance metric defined")


# Dummy test classes
class DummyExample:
    def __init__(self, score):
        self.score = score


class DummyPred:
    def __init__(self, score):
        self.score = score


# Test cases
test_gold = DummyExample(7)
print(f"Test: gold=7, pred=7 → {importance_metric(test_gold, DummyPred(7)):.2f} (expect 1.0)")
print(f"Test: gold=7, pred=8 → {importance_metric(test_gold, DummyPred(8)):.2f} (expect 0.8)")
print(f"Test: gold=7, pred=9 → {importance_metric(test_gold, DummyPred(9)):.2f} (expect 0.5)")
print(f"Test: gold=7, pred=1 → {importance_metric(test_gold, DummyPred(1)):.2f} (expect 0.0)")


✅ GEPA-compatible importance metric defined
Test: gold=7, pred=7 → 1.00 (expect 1.0)
Test: gold=7, pred=8 → 0.80 (expect 0.8)
Test: gold=7, pred=9 → 0.50 (expect 0.5)
Test: gold=7, pred=1 → 0.00 (expect 0.0)


## Create Uncompiled Baseline

In [ ]:
# Uncompiled baseline (ChainOfThought)
uncompiled_scorer = dspy.ChainOfThought(ScoreImportance)

print("✅ Uncompiled baseline created")
print("Module type:", type(uncompiled_scorer).__name__)

✅ Uncompiled baseline created
Module type: ChainOfThought


## Evaluate Uncompiled Baseline

Quick check to verify baseline matches Day 3 results (77.5% ±2 accuracy)

In [ ]:
def evaluate_module(module, testset, verbose=False):
    """Evaluate module on test set."""
    results = {
        'exact': 0,
        'within_1': 0,
        'within_2': 0,
        'errors': [],
        'predictions': []
    }

    for i, example in enumerate(testset):
        try:
            pred = module(
                observation=example.observation,
                agent_goal=example.agent_goal,
                agent_personality=example.agent_personality
            )
            pred_score = int(pred.score)
            pred_score = max(1, min(10, pred_score))  # Clamp
        except Exception as e:
            if verbose:
                print(f"Error on example {i}: {e}")
            pred_score = 5  # Default

        gold_score = int(example.score)
        error = abs(pred_score - gold_score)

        results['errors'].append(error)
        results['predictions'].append(pred_score)

        if error == 0:
            results['exact'] += 1
        if error <= 1:
            results['within_1'] += 1
        if error <= 2:
            results['within_2'] += 1

    n = len(testset)
    results['accuracy_exact'] = results['exact'] / n * 100
    results['accuracy_within_1'] = results['within_1'] / n * 100
    results['accuracy_within_2'] = results['within_2'] / n * 100
    results['mean_error'] = sum(results['errors']) / len(results['errors'])
    results['max_error'] = max(results['errors'])

    return results

print("Evaluating uncompiled baseline (this may take 2-3 minutes)...\n")
uncompiled_results = evaluate_module(uncompiled_scorer, trainset, verbose=True)

print("=" * 70)
print("UNCOMPILED BASELINE PERFORMANCE")
print("=" * 70)
print(f"Exact matches:      {uncompiled_results['exact']:2d}/40 ({uncompiled_results['accuracy_exact']:.1f}%)")
print(f"Within ±1:          {uncompiled_results['within_1']:2d}/40 ({uncompiled_results['accuracy_within_1']:.1f}%)")
print(f"Within ±2:          {uncompiled_results['within_2']:2d}/40 ({uncompiled_results['accuracy_within_2']:.1f}%)")
print(f"Mean Absolute Error: {uncompiled_results['mean_error']:.2f}")
print(f"Max Error:          {uncompiled_results['max_error']}")
print("=" * 70)

Evaluating uncompiled baseline (this may take 2-3 minutes)...



2025/10/11 23:52:04 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/10/11 23:52:15 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


UNCOMPILED BASELINE PERFORMANCE
Exact matches:       9/40 (22.5%)
Within ±1:          25/40 (62.5%)
Within ±2:          31/40 (77.5%)
Mean Absolute Error: 1.48
Max Error:          5


## Initialize GEPA Optimizer

In [ ]:
from dspy.teleprompt import GEPA
import time
import os
import dspy

# Configure reflection LM (uses Together.ai model)
reflection_lm = dspy.LM(
    model="together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    api_key=os.getenv("TOGETHER_API_KEY"),
    temperature=0.5  # Slightly higher temp for creative reflection
)

# GEPA optimizer with correct configuration
optimizer = GEPA(
    metric=importance_metric,
    auto="medium",               # Use preset budget ("light", "medium", or "heavy")
    reflection_minibatch_size=5, # Number of examples per reflection round
    track_stats=True,            # Enable detailed logging
    reflection_lm=reflection_lm  # Required for GEPA
)

print("✅ GEPA optimizer initialized")
print("Budget level: medium")


✅ GEPA optimizer initialized
Budget level: medium


## Run GEPA Compilation

**This cell will take 4-6 hours to run**  
You can leave the tab open or use a keep-alive script  
Checkpoints saved every 10 iterations to Google Drive

In [ ]:
print("=" * 70)
print("STARTING GEPA COMPILATION")
print("=" * 70)
print(f"Training set size: {len(trainset)}")
print(f"Budget: 40 rollouts")
print(f"Expected runtime: 4-6 hours")
print(f"Start time: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)
print("\n Compilation running... (this will take a while)\n")

start_time = time.time()

# Run compilation
try:
    compiled_scorer = optimizer.compile(
        uncompiled_scorer,
        trainset=trainset
    )

    elapsed = time.time() - start_time
    print("\n" + "=" * 70)
    print("COMPILATION COMPLETE!")
    print("=" * 70)
    print(f"Time elapsed: {elapsed/3600:.2f} hours ({elapsed/60:.1f} minutes)")
    print(f"End time: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 70)

except Exception as e:
    print(f"\n Compilation failed: {e}")
    print("\nTroubleshooting steps:")
    print("2. Check internet connection")
    print("3. Try reducing budget to 30")
    print("4. Try MIPROv2 optimizer as fallback")
    raise

2025/10/12 00:17:40 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 890 metric calls of the program. This amounts to 22.25 full evals on the train set.
2025/10/12 00:17:40 INFO dspy.teleprompt.gepa.gepa: Using 40 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.


STARTING GEPA COMPILATION
Training set size: 40
Budget: 40 rollouts
Expected runtime: 4-6 hours
Start time: 2025-10-12 00:17:40

 Compilation running... (this will take a while)



GEPA Optimization:   0%|          | 0/890 [00:00<?, ?rollouts/s]2025/10/12 00:18:12 INFO dspy.evaluate.evaluate: Average Metric: 24.6 / 40 (61.5%)
2025/10/12 00:18:12 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.615
GEPA Optimization:   4%|▍         | 40/890 [00:32<11:23,  1.24rollouts/s]2025/10/12 00:18:12 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.615


Average Metric: 3.60 / 5 (72.0%): 100%|██████████| 5/5 [00:00<00:00, 660.56it/s]

2025/10/12 00:18:12 INFO dspy.evaluate.evaluate: Average Metric: 3.6 / 5 (72.0%)


...

2025/10/12 00:33:15 INFO dspy.teleprompt.gepa.gepa: Iteration 37: Proposed new text for predict: # Task Instructions

## Task Name: Rating the Importance of Observations for Agent Goals

## Task Description:
The goal of this task is to rate the importance of an observation for an agent's goals, given the observation, the agent's goal, and the agent's personality. The rating should be based on how directly the observation impacts the agent's plans, goals, or personality, and should be scored on a scale of 1-10.

## Input Format:
The input will consist of three components:
- `observation`: a string describing an event or situation that the agent has encountered.
- `agent_goal`: a string describing the primary goal of the agent.
- `agent_personality`: a string describing the personality traits of the agent, including their values, motivations, and behaviors.

## Task Requirements:
1. The assistant should carefully read the input components and generate a reasoning statement explaining why

Average Metric: 2.50 / 5 (50.0%): 100%|██████████| 5/5 [00:00<00:00, 742.09it/s]

2025/10/12 00:33:16 INFO dspy.evaluate.evaluate: Average Metric: 2.5 / 5 (50.0%)


2025/10/12 00:33:18 INFO dspy.teleprompt.gepa.gepa: Iteration 38: Proposed new text for predict: python
# Task Instructions

## Task Description
The task involves rating the importance of a given observation for an agent's goals. The agent has a specific goal and personality, and the observation is presented as a piece of information that may be relevant to the agent's goal. This task requires the assistant to consider the agent's personality traits, their goal, and the observation itself to assign a score based on its relevance to the agent's goal.

## Input Format
The input consists of three components:
- `observation`: A piece of information that may be relevant to the agent's goal.
- `agent_goal`: The agent's primary goal, which is a string describing what the agent is trying to achieve.
- `agent_personality`: A dictionary describing the agent's personality traits, which may influence their behavior and decision-making.

## Domain-Specific Factual Information
Based on the provided 


COMPILATION COMPLETE!
Time elapsed: 0.26 hours (15.7 minutes)
End time: 2025-10-12 00:33:22


## Evaluate Compiled Module

In [ ]:
print("Evaluating compiled module (this may take 2-3 minutes)...\n")
compiled_results = evaluate_module(compiled_scorer, trainset, verbose=True)

print("=" * 70)
print("COMPILED MODULE PERFORMANCE")
print("=" * 70)
print(f"Exact matches:      {compiled_results['exact']:2d}/40 ({compiled_results['accuracy_exact']:.1f}%)")
print(f"Within ±1:          {compiled_results['within_1']:2d}/40 ({compiled_results['accuracy_within_1']:.1f}%)")
print(f"Within ±2:          {compiled_results['within_2']:2d}/40 ({compiled_results['accuracy_within_2']:.1f}%)")
print(f"Mean Absolute Error: {compiled_results['mean_error']:.2f}")
print(f"Max Error:          {compiled_results['max_error']}")
print("=" * 70)

Evaluating compiled module (this may take 2-3 minutes)...

COMPILED MODULE PERFORMANCE
Exact matches:      12/40 (30.0%)
Within ±1:          26/40 (65.0%)
Within ±2:          33/40 (82.5%)
Mean Absolute Error: 1.38
Max Error:          5


## Comparison: Uncompiled vs Compiled

In [ ]:
print("\n" + "=" * 70)
print("PERFORMANCE COMPARISON")
print("=" * 70)

improvement = compiled_results['accuracy_within_2'] - uncompiled_results['accuracy_within_2']
mae_improvement = uncompiled_results['mean_error'] - compiled_results['mean_error']

print("\n| Metric | Uncompiled | Compiled | Improvement |")
print("|--------|------------|----------|-------------|")
print(f"| Exact  | {uncompiled_results['accuracy_exact']:5.1f}%   | {compiled_results['accuracy_exact']:5.1f}% | {compiled_results['accuracy_exact'] - uncompiled_results['accuracy_exact']:+6.1f}% |")
print(f"| ±1     | {uncompiled_results['accuracy_within_1']:5.1f}%   | {compiled_results['accuracy_within_1']:5.1f}% | {compiled_results['accuracy_within_1'] - uncompiled_results['accuracy_within_1']:+6.1f}% |")
print(f"| **±2** | **{uncompiled_results['accuracy_within_2']:5.1f}%** | **{compiled_results['accuracy_within_2']:5.1f}%** | **{improvement:+6.1f}%** |")
print(f"| MAE    | {uncompiled_results['mean_error']:5.2f}    | {compiled_results['mean_error']:5.2f}  | {mae_improvement:+6.2f}   |")

print("\n" + "=" * 70)

# Success criteria check
if improvement >= 10:
    print("\n SUCCESS! Improvement ≥10%, proceed to Day 5")
    print(f"   Target: 80% → Achieved: {compiled_results['accuracy_within_2']:.1f}%")
elif improvement >= 5:
    print("\n  PARTIAL SUCCESS. Improvement 5-10%, consider iteration")
    print(f"   Target: 80% → Achieved: {compiled_results['accuracy_within_2']:.1f}%")
else:
    print("\n INSUFFICIENT IMPROVEMENT (<5%)")
    print("\n   Options:")
    print("   1. Try MIPROv2 optimizer")
    print("   2. Add more mundane category seeds")
    print("   3. Adjust metric tolerances")
    print("   4. Review worst-performing seeds")


PERFORMANCE COMPARISON

| Metric | Uncompiled | Compiled | Improvement |
|--------|------------|----------|-------------|
| Exact  |  22.5%   |  30.0% |   +7.5% |
| ±1     |  62.5%   |  65.0% |   +2.5% |
| **±2** | ** 77.5%** | ** 82.5%** | **  +5.0%** |
| MAE    |  1.48    |  1.38  |  +0.10   |


  PARTIAL SUCCESS. Improvement 5-10%, consider iteration
   Target: 80% → Achieved: 82.5%


## Analyze by Category

In [ ]:
from collections import defaultdict

def evaluate_by_category(module, testset):
    """Break down performance by category."""
    category_results = defaultdict(lambda: {'errors': [], 'gold_scores': [], 'pred_scores': []})

    for example in testset:
        try:
            pred = module(
                observation=example.observation,
                agent_goal=example.agent_goal,
                agent_personality=example.agent_personality
            )
            pred_score = max(1, min(10, int(pred.score)))
        except Exception:
            pred_score = 5

        gold_score = int(example.score)
        error = abs(pred_score - gold_score)

        cat = example.category
        category_results[cat]['errors'].append(error)
        category_results[cat]['gold_scores'].append(gold_score)
        category_results[cat]['pred_scores'].append(pred_score)

    return category_results

print("\n" + "=" * 70)
print(" UNCOMPILED MODULE BY CATEGORY")
print("=" * 70)

uncompiled_by_cat = evaluate_by_category(uncompiled_scorer, trainset)
for cat in sorted(uncompiled_by_cat.keys()):
    data = uncompiled_by_cat[cat]
    within_2 = sum(1 for e in data['errors'] if e <= 2)
    accuracy = within_2 / len(data['errors']) * 100
    mae = sum(data['errors']) / len(data['errors'])
    print(f"{cat:20s}: {accuracy:5.1f}% ±2 accuracy, MAE={mae:.2f} ({len(data['errors'])} seeds)")

print("\n" + "=" * 70)
print(" COMPILED MODULE BY CATEGORY")
print("=" * 70)

compiled_by_cat = evaluate_by_category(compiled_scorer, trainset)
for cat in sorted(compiled_by_cat.keys()):
    data = compiled_by_cat[cat]
    within_2 = sum(1 for e in data['errors'] if e <= 2)
    accuracy = within_2 / len(data['errors']) * 100
    mae = sum(data['errors']) / len(data['errors'])

    # Get improvement
    uncompiled_data = uncompiled_by_cat[cat]
    uncompiled_within_2 = sum(1 for e in uncompiled_data['errors'] if e <= 2)
    uncompiled_accuracy = uncompiled_within_2 / len(uncompiled_data['errors']) * 100
    improvement = accuracy - uncompiled_accuracy

    print(f"{cat:20s}: {accuracy:5.1f}% ±2 accuracy, MAE={mae:.2f} ({len(data['errors'])} seeds) [{improvement:+5.1f}%]")


 UNCOMPILED MODULE BY CATEGORY
edge_cases          :  66.7% ±2 accuracy, MAE=2.00 (6 seeds)
emotional           :  66.7% ±2 accuracy, MAE=1.67 (6 seeds)
environmental       :  83.3% ±2 accuracy, MAE=1.33 (6 seeds)
goal_relevant       : 100.0% ±2 accuracy, MAE=0.62 (8 seeds)
mundane             :   0.0% ±2 accuracy, MAE=4.17 (6 seeds)
social              : 100.0% ±2 accuracy, MAE=0.75 (8 seeds)

 COMPILED MODULE BY CATEGORY
edge_cases          : 100.0% ±2 accuracy, MAE=0.83 (6 seeds) [+33.3%]
emotional           :  66.7% ±2 accuracy, MAE=2.17 (6 seeds) [ +0.0%]
environmental       : 100.0% ±2 accuracy, MAE=1.33 (6 seeds) [+16.7%]
goal_relevant       : 100.0% ±2 accuracy, MAE=0.75 (8 seeds) [ +0.0%]
mundane             :  16.7% ±2 accuracy, MAE=2.83 (6 seeds) [+16.7%]
social              : 100.0% ±2 accuracy, MAE=0.75 (8 seeds) [ +0.0%]


## Top Errors Analysis

In [ ]:
# Find worst predictions
error_details = []
for i, example in enumerate(trainset):
    error = compiled_results['errors'][i]
    pred_score = compiled_results['predictions'][i]
    gold_score = int(example.score)

    error_details.append({
        'seed_id': example.seed_id,
        'category': example.category,
        'observation': example.observation[:60] + '...',
        'gold': gold_score,
        'pred': pred_score,
        'error': error
    })

# Sort by error descending
error_details.sort(key=lambda x: x['error'], reverse=True)

print("\n" + "=" * 70)
print(" TOP 10 LARGEST ERRORS (COMPILED)")
print("=" * 70)

for i, err in enumerate(error_details[:10], 1):
    print(f"\n{i}. Seed #{err['seed_id']}: Error = {err['error']}")
    print(f"   Observation: \"{err['observation']}\"")
    print(f"   Gold: {err['gold']}, Predicted: {err['pred']}")
    print(f"   Category: {err['category']}")


 TOP 10 LARGEST ERRORS (COMPILED)

1. Seed #15: Error = 5
   Observation: "Fire alarm going off in the building!..."
   Gold: 10, Predicted: 5
   Category: emotional

2. Seed #17: Error = 5
   Observation: "I just received devastating news about a family member..."
   Gold: 10, Predicted: 5
   Category: emotional

3. Seed #12: Error = 4
   Observation: "Someone's phone buzzed far away..."
   Gold: 1, Predicted: 5
   Category: mundane

4. Seed #14: Error = 4
   Observation: "A bird chirped in the distance..."
   Gold: 1, Predicted: 5
   Category: mundane

5. Seed #10: Error = 3
   Observation: "A clock is ticking in the background..."
   Gold: 2, Predicted: 5
   Category: mundane

6. Seed #11: Error = 3
   Observation: "The sky is blue and cloudless..."
   Gold: 2, Predicted: 5
   Category: mundane

7. Seed #13: Error = 3
   Observation: "The streetlight turned on at dusk..."
   Gold: 2, Predicted: 5
   Category: mundane

8. Seed #2: Error = 2
   Observation: "Bob said he's too busy to

## Save Compiled Program

In [ ]:
# Save to Google Drive
save_path = '/content/drive/MyDrive/mini-town/compiled/compiled_scorer.json'
compiled_scorer.save(save_path)
print(f" Compiled scorer saved to: {save_path}")

# Extract and save prompts for inspection
prompt_text = str(compiled_scorer.dump_state())
prompt_path = '/content/drive/MyDrive/mini-town/compiled/prompt_scorer.txt'
with open(prompt_path, 'w') as f:
    f.write(prompt_text)
print(f" Prompts saved to: {prompt_path}")

# Save results summary
results_summary = {
    'compilation_time_hours': elapsed / 3600,
    'uncompiled': {
        'accuracy_within_2': uncompiled_results['accuracy_within_2'],
        'mean_error': uncompiled_results['mean_error'],
        'exact_matches': uncompiled_results['exact']
    },
    'compiled': {
        'accuracy_within_2': compiled_results['accuracy_within_2'],
        'mean_error': compiled_results['mean_error'],
        'exact_matches': compiled_results['exact']
    },
    'improvement': {
        'accuracy_delta': improvement,
        'mae_delta': mae_improvement
    }
}

results_path = '/content/drive/MyDrive/mini-town/compiled/compilation_results.json'
with open(results_path, 'w') as f:
    json.dump(results_summary, f, indent=2)
print(f" Results summary saved to: {results_path}")

print("\n" + "=" * 70)
print(" COMPILATION COMPLETE!")
print("=" * 70)
print("\nNext steps:")
print("1. Download compiled_scorer.json to local project")
print("2. Review prompt_scorer.txt to understand optimizations")
print("3. Proceed to Day 5: A/B testing in full simulation")

 Compiled scorer saved to: /content/drive/MyDrive/mini-town/compiled/compiled_scorer.json
 Prompts saved to: /content/drive/MyDrive/mini-town/compiled/prompt_scorer.txt
 Results summary saved to: /content/drive/MyDrive/mini-town/compiled/compilation_results.json

 COMPILATION COMPLETE!

Next steps:
1. Download compiled_scorer.json to local project
2. Review prompt_scorer.txt to understand optimizations
3. Proceed to Day 5: A/B testing in full simulation


## Fallback: MIPROv2 (if GEPA doesn't work)

Uncomment and run this cell if GEPA has issues

In [ ]:
# from dspy.optimizers import MIPROv2
#
# print("Falling back to MIPROv2 optimizer...")
#
# mipro_optimizer = MIPROv2(
#     metric=importance_metric,
#     auto="medium",
#     num_trials=10,
#     max_bootstrapped_demos=4,
#     max_labeled_demos=5
# )
#
# print("Running MIPROv2 compilation (expect 6-8 hours)...")
# start_time = time.time()
#
# compiled_scorer_mipro = mipro_optimizer.compile(
#     uncompiled_scorer,
#     trainset=trainset
# )
#
# elapsed = time.time() - start_time
# print(f"\n MIPROv2 compilation complete! Time: {elapsed/3600:.2f} hours")
#
# # Evaluate MIPROv2 results
# mipro_results = evaluate_module(compiled_scorer_mipro, trainset)
# print(f"MIPROv2 ±2 accuracy: {mipro_results['accuracy_within_2']:.1f}%")
# print(f"Improvement: +{mipro_results['accuracy_within_2'] - uncompiled_results['accuracy_within_2']:.1f}%")